# QA Chatbot
# Structure

<br/>*{개발중: 완료 X, 디벨롭 필요: 완성O[보수 필요], 주석없음: 완성 O}*
<br/>chatbot
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├ config
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├ train_tools
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;│&nbsp;&nbsp;&nbsp;└ dict
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├ utils
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├ model
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;│&nbsp;&nbsp;&nbsp;├ intent
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;│&nbsp;&nbsp;&nbsp;└ ner
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├ test
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sample

In [ ]:
!pip install tensorflow
!pip install keras
!pip install konlpy
!pip install seqeval

#Config

In [ ]:
# ./config/GlobalPrams.py

# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = 25

def GlobalParams():
    	global MAX_SEQ_LEN

#Utils

In [ ]:
#Preprocess
from konlpy.tag import Komoran
import pickle

class Preprocess:
    # 생성자
	def __init__(self, word2index_dic="", userdic=None):
		# 단어 인덱스 사전 불러오기
		if word2index_dic != "":
			f = open(word2index_dic, "rb")
			self.word_index = pickle.load(f)
			f.close()
		else:
			self.word_index = None

		# 형태소 분석기 초기화
		self.komoran = Komoran(userdic=userdic)

		# 제외할 품사
		# 참조: https://docs.komoran.kr/firststep/postypes.html
		self.exclusion_tags = [
            "VV", "VA", "VX", "VCP", "VCN", # 용언 제거
			"JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ", "JX", "JC", # 관계언 제거
			"SF", "SP", "SS", "SE", "SO", # 기호 제거
			"EP", "EF", "EC", "ETN", "ETM", # 어미 제거
			"XSN", "XSV", "XSA", # 접미사 제거
		]

	# 형태소 분석기 POS tagger (래퍼 함수)
	def pos(self, sentence):
		return self.komoran.pos(sentence)

	# 불용어 제거 후 필요한 품사 정보만 가져오기
	def get_keywords(self, pos, without_tag=False):
		f = lambda x: x in self.exclusion_tags
		word_list = []
		for p in pos:
			if f(p[1]) is False: # 불용어 리스트에 없는 경우에만 저장
				word_list.append(p if without_tag is False else p[0])
		return word_list

	# 키워드를 단어 인덱스 시퀀스로 변환
	def get_wordidx_sequence(self, keywords):
		if self.word_index is None:
			return []
		w2i = []
		for word in keywords:
			try:
				w2i.append(self.word_index[word])
			except KeyError:
				# 해당 단어가 사전에 없는 경우 OOV 처리
				w2i.append(self.word_index["OOV"])
		return w2i

#Cal_padding

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # 0 = all messages are logged (default behavior), 1 = INFO messages are not printed, 2 = INFO and WARNING messages are not printed, 3 = INFO, WARNING, and ERROR messages are not printed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Komoran

from utils.Preprocess import Preprocess

data = pd.read_csv('./model/intent/train_data.csv')

# 전처리 객체 생성
p = Preprocess(userdic='./utils/user_dic.tsv')

data_tokenized = [p.pos(text_) for text_ in data['query']]
# pos = data_tokenized[0]
# keywords = p.get_keywords(pos, without_tag=True)

data_list = []
for i in range(len(data_tokenized)):
    pos = data_tokenized[i]
    keywords = p.get_keywords(pos, without_tag=True)
    data_list.append(keywords)

num_tokens = [len(tokens) for tokens in data_list]
num_tokens = np.array(num_tokens)

# 평균값, 최댓값, 표준편차
print(f"토큰 길이 평균: {np.mean(num_tokens)}")
print(f"토큰 길이 최대: {np.max(num_tokens)}")
print(f"토큰 길이 표준편차: {np.std(num_tokens)}")

# plt.title('all text length')
# plt.hist(num_tokens, bins=100)
# plt.xlabel('length of samples')
# plt.ylabel('number of samples')
# plt.show()

select_length = 17

def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1

    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %0.2f%%'%(max_len, (cnt / len(nested_list) * 100)))

below_threshold_len(select_length, data_list)

#Dict

In [ ]:
# creat_dict.py

# 단어 사전 파일 생성
# 챗봇에 사용하는 사전 파일

import os
from konlpy.tag import Komoran

from utils.Preprocess import Preprocess
from tensorflow.keras import preprocessing
import pickle
import pandas as pd

# 말뭉치 데이터 읽어오기
test_data = pd.read_csv('test_data.csv')
test_data.dropna(inplace=True)  # NaN 데이터 제거, inplace: 원본을 변경할지의 여부
corpus_data = list(test_data['Query']) + list(test_data['Answer'])

# 전처리 객체 생성
p = Preprocess(userdic='./user_dic.tsv')

# 말뭉치 데이터로부터 사전 리스트 생성
dict = []
for c in corpus_data:
    pos = p.pos(c)
    for k in pos:
        dict.append(k[0])
print(dict)

# # 사전에 사용될 word2index 생성
# # 사전의 첫 번째 인덱스에는 OVV 사용
# tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')#, num_words=100000)
# tokenizer.fit_on_texts(dict)
# word_index = tokenizer.word_index

# # 사전 파일 생성
# f = open("chatbot_dict_test_v0.1.1.bin", "wb")
# try:
#     pickle.dump(word_index, f)
# except Exception as e:
#     print(e)
# finally:
#     f.close()

# import json
# with open('chatbot_dict_test_v0.1.1.json', 'w', encoding='utf-8') as f:
#     json.dump(word_index, f, indent=4, ensure_ascii=False)

ModuleNotFoundError: ignored

In [ ]:
# create_dict.py

# 단어 사전 파일 생성
# 챗봇에 사용하는 사전 파일

import os
from konlpy.tag import Komoran

# from utils.Preprocess import Preprocess
from tensorflow.keras import preprocessing
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 말뭉치 데이터 읽어오기
test_data = pd.read_csv('test_data.csv')
test_data.dropna(inplace=True)  # NaN 데이터 제거, inplace: 원본을 변경할지의 여부
corpus_data = list(test_data['Query']) + list(test_data['Answer'])

# 전처리 객체 생성
p = Preprocess(userdic='user_dic.tsv')

# 말뭉치 데이터로부터 사전 리스트 생성
dict = []
for c in corpus_data:
    pos = p.pos(c)
    for k in pos:
        dict.append(k[0])

num_tokens = [len(tokens) for tokens in dict]
num_tokens = np.array(num_tokens)

# 평균값, 최댓값, 표준편차
print(f"토큰 길이 평균: {np.mean(num_tokens)}")
print(f"토큰 길이 최대: {np.max(num_tokens)}")
print(f"토큰 길이 표준편차: {np.std(num_tokens)}")

# # 사전에 사용될 word2index 생성
# # 사전의 첫 번째 인덱스에는 OVV 사용
# tokenizer = preprocessing.text.Tokenizer(oov_token='OOV', num_words=100000)
# tokenizer.fit_on_texts(dict)
# word_index = tokenizer.word_index

# # 사전 파일 생성
# f = open("chatbot_dict_test_v0.1.bin", "wb")
# try:
#     pickle.dump(word_index, f)
# except Exception as e:
#     print(e)
# finally:
#     f.close()

# import json
# with open('chatbot_dict_test_v0.1.json', 'w', encoding='utf-8') as f:
#     json.dump(word_index, f, indent=4, ensure_ascii=False)

ModuleNotFoundError: ignored

In [ ]:
from tensorflow.keras import preprocessing
import pickle

# 말뭉치 데이터 읽어오기
def read_corpus_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # 헤더 제거
  return data


# 말뭉치 데이터 가져오기
corpus_data = read_corpus_data('corpus.txt')

# 말뭉치 데이터에서 키워드만 추출해서 사전 리스트 생성
p = Preprocess()
dict= []
for c in corpus_data:
  pos = p.pos(c[1])
  for k in pos:
    dict.append(k[0])

# 사전에 사용될 word2index 생성
# 사전의 첫 번째 인덱스에는 OOV 사용
tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(dict)
word_index = tokenizer.word_index

#사전 파일 생성
f = open("chatbot_dict.bin", "wb")
try:
  pickle.dump(word_index, f)
except Exception as e:
  print(e)
finally:
  f.close()

#Model

intent

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 의도 분류 모델 모듈
class IntentModel:
    def __init__(self, model_name, preprocess):

        # 의도 클래스 별 레이블
        self.labels = {0: "공통", 1: "철도차량", 2: "철도용품", 3: "철도차량개조"}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = preprocess


    # 의도 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 단어 시퀀스 벡터 크기
        # from config.GlobalParams import MAX_SEQ_LEN

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]

Intent_Model_Train

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

from utils.Preprocess import Preprocess
from config.GlobalParams import MAX_SEQ_LEN

# 데이터 읽어오기
train_file = './model/intent/train_data.csv'
data = pd.read_csv(train_file)
query = data['query'].tolist()
intent = data['intent'].tolist()

p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict_test_v0.1.bin',
               userdic='./utils/user_dic.tsv')

# 단어 시퀀스 생성
sequences = []
for sentence in query:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

# 단어 시퀀스 패딩
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')


# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intent))
ds = ds.shuffle(len(query))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(2)
val_ds = ds.skip(train_size).take(val_size).batch(2)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(1)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 3
VOCAB_SIZE = len(p.word_index) + 1

# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(4, name='logits')(dropout_hidden)
predictions = Dense(4, activation=tf.nn.softmax)(logits)

#모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
# fit함수를 돌리면 이렇게 아래 출력창에 훈련하는 과정이 출력된다.
# partial_x_train, partial_y_train / 트레이닝 데이터 중에서 훈련을 시키는 데이터 수
# epoch / 반복 훈련 수
# batch_size=512 / 512 번 마다 가중치를 업데이트
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)


# 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))


# 모델 저장
model.save('intent_model_v.0.1.h5')

ModuleNotFoundError: ignored

ner

In [ ]:
# train.model
import sys, os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))))
# from utils.Preprocess import Preprocess

# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

#../../train_tools/dict/chatbot_dict_test_v0.1.bin
#../../utils/user_dic.tsv

p = Preprocess(word2index_dic='chatbot_dict.bin',
               userdic='sample_user_dic.tsv')

# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('ner_train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)


print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

# 토크나이저 정의
# p = preprocessing.text.Tokenizer(oov_token='OOV')
# p.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

# 단어사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
# x_train = p.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환 하기 위해 사용
index_to_ner[0] = 'PAD'

# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=.2,
                                                    random_state=1234)

# 출력 데이터를 one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)


# 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save('ner_model_test_v.0.2.h5')

# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result

# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

model = tf.keras.models.load_model('ner_model_test_v.0.2.h5')

# 새로운 유형의 문장 NER 예측
word_to_index = tag_tokenizer.word_index
new_sentence = '오늘 13시 30분 짜장면 주문할게요.'
pos = p.pos(new_sentence)
keywords = p.get_keywords(pos, without_tag=True)
new_seq = p.get_wordidx_sequence(keywords)
print("새로운 유형의 시퀀스 : ", new_seq)

new_x = []
for w in keywords:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        new_x.append(word_to_index['OOV'])
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0, maxlen=max_len)
print("새로운 유형의 시퀀스 : ", new_x)
print("새로운 유형의 시퀀스 : ", new_padded_seqs)

# from tensorflow.keras.models import Model, load_model
# NER 예측
# model = load_model('ner_model_test.h5')
mp = model.predict(np.array([new_padded_seqs[0]]))
mp = np.argmax(mp, axis=-1)   # 예측된 NER 인덱스값 추출

print("{:10} {:5}".format("단어", "예측된 NER"))
print("-"*50)
for w, pred in zip(keywords, mp[0]):
    print("{:10} {:5}".format(w, index_to_ner[pred]))

샘플 크기 : 
 61999
0번 째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번 째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 168
샘플 단어 시퀀스 평균 길이 : 8.796238649010467
BIO 태그 사전 크기 : 10
단어 사전 크기 : 17751
학습 샘플 시퀀스 형상 :  (49599, 40)
학습 샘플 레이블 형상 :  (49599, 40, 10)
테스트 샘플 시퀀스 형상 :  (12400, 40)
테스트 샘플 레이블 형상 :  (12400, 40, 10)
Epoch 1/10
388/388 [==============================] - 381s 919ms/step - loss: 0.1131 - accuracy: 0.9684
Epoch 2/10
388/388 [==============================] - 366s 945ms/step - loss: 0.0384 - accuracy: 0.9875
Epoch 3/10
388/388 [==============================] - 364s 938ms/step - loss: 0.0256 - accuracy: 0.9916
Epoch 4/10
388/388 [==============================] - 358s 922ms/step - loss: 0.0204 - accuracy: 0.9931
Epoch 5/10
388/388 [==============================] - 359s 925ms/step - loss: 0.0172 - accuracy: 0.9943
Epoch 6/10
388/388 [==============================] - 359s 926ms/step - loss: 0.0149 - accuracy: 0.9951
Epoch 7/10
388/388 [===============

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:1

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       303
           _       0.55      0.53      0.54       658
         _DT       1.00      1.00      1.00     13683
       _FOOD       1.00      1.00      1.00     11655
         _LC       0.78      0.52      0.62       314
         _OG       0.57      0.55      0.56       460
         _PS       0.60      0.50      0.54       396
         _TI       0.80      0.70      0.75        61

   micro avg       0.97      0.97      0.97     27530
   macro avg       0.79      0.73      0.75     27530
weighted avg       0.97      0.97      0.97     27530

F1-score: 97.0%
새로운 유형의 시퀀스 :  [40, 1, 1, 527, 3]
새로운 유형의 시퀀스 :  [1, 1, 1, 1, 1]
새로운 유형의 시퀀스 :  [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
1/1 [==============================] - 1s 888ms/step
단어         예측된 NER
--------------------------------------------------
오늘         B_OG 
13시        O    
30분       

In [ ]:
#ner model test
import os, sys
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
import numpy as np
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))))
# from utils.Preprocess import Preprocess


#../../train_tools/dict/chatbot_dict.bin
#../../utils/user_dic.tsv

p = Preprocess(word2index_dic='chatbot_dict.bin',
               userdic='sample_user_dic.tsv')
# 토크나이저 정의
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.

# 새로운 유형의 문장 NER 예측
new_sentence = '무지개떡 주문 하고 싶어요'
pos = p.pos(new_sentence)
keywords = p.get_keywords(pos, without_tag=True)
new_seq = p.get_wordidx_sequence(keywords)
print("새로운 유형의 시퀀스 : ", new_seq)

new_padded_seqs = preprocessing.sequence.pad_sequences([new_seq], padding='post', value=0, maxlen=40)
print("새로운 유형의 시퀀스 : ", new_seq)
print("새로운 유형의 시퀀스 : ", new_padded_seqs)

# from tensorflow.keras.models import Model, load_model
# NER 예측
model = load_model('ner_model_test_v.0.2.h5')
mp = model.predict(np.array([new_padded_seqs[0]]))
mp = np.argmax(mp, axis=-1)   # 예측된 NER 인덱스값 추출

print("{:10} {:5}".format("단어", "예측된 NER"))
print("-"*50)
index_to_ner = {1: 'O', 2: 'B_DT', 3: 'B_FOOD', 4: 'I', 5: 'B_OG', 6: 'B_PS', 7: 'B_LC', 8: 'NNP', 9: 'B_TI', 0: 'PAD'}
for w, pred in zip(keywords, mp[0]):
    print("{:10} {:5}".format(w, index_to_ner[pred]))

새로운 유형의 시퀀스 :  [1, 3]
새로운 유형의 시퀀스 :  [1, 3]
새로운 유형의 시퀀스 :  [[1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
1/1 [==============================] - 1s 1s/step
단어         예측된 NER
--------------------------------------------------
무지개떡       B_FOOD
주문         O    


#Test

In [ ]:
# test/model_intent_test.py

# from utils.Preprocess import Preprocess
# from model.intent.IntentModel import IntentModel
import pandas as pd


#./train_tools/dict/chatbot_dict_test_v0.1.bin
#./utils/user_dic.tsv

p = Preprocess(word2index_dic='chatbot_dict_test_v.0.1.bin',
               userdic='user_dic.tsv')

#./model/intent/intent_model_v.0.1.h5
intent = IntentModel(model_name='intent_model_v.0.1.h5', preprocess=p)

#./model/intent/train_data.csv
train_file = 'train_data.csv'
data = pd.read_csv(train_file)
query = data['query'].tolist()

# for sentence in query:
#     predict = intent.predict_class(sentence)
#     predict_label = intent.labels[predict]
#     print("="*30)
#     print(sentence)
#     print("의도 예측 클래스 : ", predict)
#     print("의도 예측 레이블 : ", predict_label)

query = "미입회로 형식시험을 진행할 수 있나요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
# print(query, predict_label)
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

query = "동일한 철도차량의 계약 건마다 새로운 형식승인을 받아야 하나요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

query = "철도용품 입찰에 참여하기 위해서는 형식승인과 제작자승인을 모두 받아야 하며 비용은 제작자가 부담해야 하나요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

query = "운행승인 신청시 준비해야할 안전성 검토자료는 구체적으로 무엇인가요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

query = "철도용품 기술기준 고시 후에 종전 구매한 유지보수품은 사용가능한가요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print("="*30)
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 193ms/step
미입회로 형식시험을 진행할 수 있나요?
의도 예측 클래스 :  0
의도 예측 레이블 :  공통
1/1 [==============================] - 0s 32ms/step
동일한 철도차량의 계약 건마다 새로운 형식승인을 받아야 하나요?
의도 예측 클래스 :  0
의도 예측 레이블 :  공통
1/1 [==============================] - 0s 44ms/step
철도용품 입찰에 참여하기 위해서는 형식승인과 제작자승인을 모두 받아야 하며 비용은 제작자가 부담해야 하나요?
의도 예측 클래스 :  2
의도 예측 레이블 :  철도용품
1/1 [==============================] - 0s 32ms/step
운행승인 신청시 준비해야할 안전성 검토자료는 구체적으로 무엇인가요?
의도 예측 클래스 :  3
의도 예측 레이블 :  철도차량개조
1/1 [==============================] - 0s 34ms/step
철도용품 기술기준 고시 후에 종전 구매한 유지보수품은 사용가능한가요?
의도 예측 클래스 :  2
의도 예측 레이블 :  철도용품


#NER_Model_Module

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 개체명 인식 모델 모듈
class NerModel:
  def __init__(self, model_name, preprocess):
    self.index_to_ner  = {1: 'O', 2: 'B_DT', 3: 'B_FOOD', 4: 'I', 5: 'B_OG', 6: 'B_PS', 7: 'B_LC', 8: 'NNP', 9: 'B_TI', 0: 'PAD'}

    # 의도 분류 모델 불러오기
    self.model = load_model(model_name)

    #챗봇 Preprocess 객체
    self.p = preprocess

  #개체명 클래스 예측
  def predict(self, query):
    #형태소 분석
    pos = self.p.pos(query)

    #문장 내 키워드 추출(불용어 제거)
    keywords = self.p.get_keywords(pos, without_tag=True)
    sequences = [self.p.get_wordidx_sequence(keywords)]

    #패딩 처리
    max_len = 40
    padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding='post', value=0, maxlen=max_len)

    #키워드별 개체명 예측
    predict = self.model.predict(np.array([padded_seqs[0]]))
    predict_class = tf.math.argmax(predict, axis=-1)

    tags = [self.index_to_ner[i] for i in predict_class.numpy()[0]]
    return list(zip(keywords, tags))

  def predict_tags(self, query):
    #형태소 분석
    pos = self.p.pos(query)

    #문장 내 키워드 추출(불용어 제거)
    keywords = self.p.get_keywords(pos, without_tag=True)
    sequences = [self.p.get_wordidx_sequence(keywords)]

    # 패딩 처리
    max_len = 40
    padded_seqs = preprocessing.seqence.pad_sequences(sequences, padding='post', value=0, max_len=max_len)

    predict = self.model.predict(np.array([padded_seqs[0]]))
    predict_class = tf.math.argmax(predict, axis=-1)

    tags = []
    for tag_idx in predict_class.numpy()[0]:
      if tag_idx == 1: continue
      tags.append(self.index_to_ner[tag_idx])

      if len(tags) == 0:
        return None
      return tags

In [ ]:
#Nermodel 객체 사용

p = Preprocess(word2index_dic='chatbot_dict.bin',
               userdic='sample_user_dic.tsv')

ner = NerModel(model_name='ner_model_test_v.0.2.h5', preprocess=p)
query = "고구마 튀김"
predicts = ner.predict(query)
print(predicts)

1/1 [==============================] - 1s 893ms/step
[('고구마', 'O'), ('튀김', 'O')]


# 답변 검색

데이터베이스 제어 모듈

In [ ]:
!pip install pymysql
!pip install logging

In [ ]:
import pymysql
import pymysql.cursors
import logging


class Database:
  '''
  데이터베이스 제어
  '''
  def __init__(self, host, user, password, db_name, charset='utf8'):
    self.host = host
    self.user = user
    self.password = password
    self.charset = charset
    self.db_name = db_name
    self.conn = None

  # DB 연결
  def connect(self):
    if self.conn != None:
      return
    self.conn = pymysql.connect(
        host=self.host,
        user=self.user,
        password=self.password,
        db=self.db_name,
        charset=self.charset,
    )

  # DB 연결 닫기
  def close(self):
    if self.conn is None:
      return
    if not self.conn.open:
      self.conn = None
      return
    self.conn.close()
    self.conn = None

  # SQL 구문 실행
  def execute(self, sql):
    last_row_id = -1
    try:
      with self.conn.cursor() as cursor:
        cursor.execute(sql)
      self.conn.commit()
      lasst_row_id = cursor.lastrowid
    except Exception as ex:
      logging.error(ex)

    finally:
      return last_row_id

  # SELECT 구문 실행 후 단 1개의 데이터 ROW만 불러옴
  def select_one(self, sql):
    result = None
    try:
      with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
        cursor.execute(sql)
        result = cursor.fetchone()
    except Exception as ex:
      logging.error(ex)

    finally:
      return result

  # SELECT 구문 실해 후 전체 데이터 ROW를 불러옴
  def select_all(self, sql):
    result = None

    try:
      with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
    except Exception as ex:
      logging.error(ex)

    finally:
      return result

챗봇 답변 검색 모듈

In [ ]:
class FindAnswer:
  def __init__(self, db):
    self.db = db

  # 검색 쿼리 생성
  def _make_query(self, intent_name, ner_tags):
    sql = "select * from chatbot_train_data"
    if intent_name != None and ner_tags == None:
      sql = sql + f" where intent={intent_name}"

    elif intent_name != None and ner_tags != None:
      where = f' where intent={intent_name}'
      if (len(ner_tags) > 0):
        where += 'and ('
        for ne in ner_tags:
          were += f' ner like {ne} or'
        where = where[:-3] + ')'
      sql = sql + where

    # 동일한 답변이 2개 이상인 경우 랜덤으로 선택
    sql = sql + " order by rand() limit 1"
    return sql


  # 답변 검색
  def search(self, intent_name, ner_tags):
    #의도명과 개체명으로 답변 검색
    sql = self._make_query(intent_name, ner_tags)
    answer = self.db.select_one(sql)

    #검색되는 답변이 없으면 의도명만 검색
    if answer is None:
      sql = self._make_query(intent_name, None)
      answer = self.db.select_one(sql)

    return (answer['answer'], answer['answer_image'])

  # NER 태그를 실제 입력된 단어로 변환
  def tag_to_word(self, ner_predicts, answer):
    for word, tag in ner_predicts:
      # 변환해야 하는 태그가 있는 경우 추가
      if tag == 'B_FOOD':
        answer = answer.replace(tag, word)

    answer = answer.replace('{', '')
    answer = answer.replace('}', '')
    return answer

#챗봇 엔진 동작